In [2]:
!type python

python is /Users/simonperrott/anaconda3/bin/python


In [3]:
import sys
sys.executable

'/Users/simonperrott/anaconda3/bin/python'

In [2]:
import numpy as np
import pandas as pd

In [7]:
pwd

'/Users/simonperrott/PycharmProjects/learning/nlp/notebooks'

In [95]:
airline_tweets = pd.read_csv('../data/twitter/training/airline_tweets.csv'
                    , usecols=['tweet_id', 'text', 'airline_sentiment'])

In [102]:
airline_tweets.head()

,tweet_id,text,sentiment
0,570306133677760513,@VirginAmerica What @dhepburn said.,0
1,570301130888122368,@VirginAmerica plus you've added commercials t...,1
2,570301083672813571,@VirginAmerica I didn't today... Must mean I n...,0
3,570301031407624196,@VirginAmerica it's really aggressive to blast...,-1
4,570300817074462722,@VirginAmerica and it's a really big bad thing...,-1


In [97]:
def standardise_sentiment_labels(column_value):
    dict_of_labels = {
        0: -1,
        "negative": -1,
        2: 0,
        "neutral": 0,
        4: +1,
        "positive": +1
    }
    return dict_of_labels[column_value]

In [103]:
airline_tweets['sentiment'] = airline_tweets.airline_sentiment.apply(standardise_sentiment_labels)
airline_tweets.drop('airline_sentiment', axis= 1, inplace=True)

AttributeError: 'DataFrame' object has no attribute 'airline_sentiment'

In [104]:
airline_tweets.groupby(airline_tweets.sentiment).count()

,tweet_id,text
sentiment,,
-1,9178,9178
0,3099,3099
1,2363,2363


In [73]:
# take the same number of positive, neutral & negative tweets into my training corpus.
balanced_tweets = []
balanced_tweets = tweets.groupby(tweets.airline_sentiment).apply(lambda x: x.sample(n=2363))
balanced_tweets = balanced_tweets.reset_index(drop=True)
balanced_tweets

,tweet_id,airline_sentiment,text
0,569644350906310657,negative,@SouthwestAir now I can't board thanks. You gu...
1,570102708734197760,negative,@JetBlue the worst flying experience I've ever...
2,569769708569767936,negative,@united Now about two dozen back in line to se...
3,569751096580812801,negative,@united doesn't help the 100 or more passenger...
4,569960010249334784,negative,@JetBlue We had some major plane-wide DirectTV...
...,...,...,...
7084,568803231335387137,positive,@united Thanks! Just DMed HN.
7085,568076081665679360,positive,@SouthwestAir OH MY GOSH SERIOUSLY?! you just ...
7086,568950850615316480,positive,@united I may not hit any new status this year...
7087,569596778426855426,positive,@SouthwestAir - just got it back about 20 mins...


In [105]:
semeval_tweets = pd.read_csv('../data/twitter/training/semeval_tweets_2016_taskA.csv'
                    , sep='\t'
                    , header=None
                    , names=['tweet_id', 'label', 'text']
                    )

In [107]:
semeval_tweets['sentiment'] = semeval_tweets.label.apply(standardise_sentiment_labels)
print(semeval_tweets.head())
semeval_tweets.drop('label', axis= 1, inplace=True)

             tweet_id     label  \
0  628949369883000832  negative   
1  628976607420645377  negative   
2  629023169169518592  negative   
3  629179223232479232  negative   
4  629186282179153920   neutral   

                                                text  sentiment  
0  dear @Microsoft the newOoffice for Mac is grea...         -1  
1  @Microsoft how about you make a system that do...         -1  
2  I may be ignorant on this issue but... should ...         -1  
3  Thanks to @microsoft, I just may be switching ...         -1  
4  If I make a game as a #windows10 Universal App...          0  


In [109]:
semeval_tweets.groupby('sentiment').count()

,tweet_id,text
sentiment,,
-1,850,850
0,2001,2001
1,3017,3017


In [116]:
leader_tweets = pd.read_csv('../data/twitter/training/labelled_leader_tweets.csv'
                           , usecols=['Id', "Text", 'Label']).rename(columns={
    "Text":"text",
    "Id":"tweet_id"
})

In [117]:
leader_tweets['sentiment'] = leader_tweets.Label.apply(standardise_sentiment_labels)
print(leader_tweets.head())
leader_tweets.drop('Label', axis= 1, inplace=True)

              tweet_id                                               text  \
0  1251509657253920773  RT @ericastarling1: One year ago today our dar...   
1  1251457526442401795  RT @simoncoveney: Last few days saw a lot of f...   
2  1251357415045500928  An incompetent political hack! Come back to Wa...   
3  1251356731478810624  This is a disgraceful situation! https://t.co/...   
4  1251356147145232385  RT @charliekirk11: This is your daily reminder...   

   Label  sentiment  
0      4          1  
1      0         -1  
2      0         -1  
3      0         -1  
4      0         -1  


In [118]:
leader_tweets.groupby('sentiment').count()

,tweet_id,text
sentiment,,
-1,33,33
0,29,29
1,37,37


In [135]:
tweets = pd.concat([leader_tweets, semeval_tweets, airline_tweets])

In [136]:
tweets.groupby('sentiment').sentiment.count()

sentiment
-1    10061
 0     5129
 1     5417
Name: sentiment, dtype: int64

In [138]:
positive_tweets = tweets[tweets.sentiment == 1]
neutral_tweets = tweets[tweets.sentiment == 0]
negative_tweets = tweets[tweets.sentiment == -1]

from sklearn.utils import shuffle

positive_tweets = shuffle(positive_tweets)[:len(neutral_tweets)]
negative_tweets = shuffle(negative_tweets)[:len(neutral_tweets)]

training_data = pd.concat([positive_tweets, neutral_tweets, negative_tweets])
training_data.groupby('sentiment').sentiment.count()

sentiment
-1    5129
 0    5129
 1    5129
Name: sentiment, dtype: int64

In [140]:
training_data.head()

,tweet_id,text,sentiment
999,641567237409783810,What a quality result that was beating Barca a...,1
11211,568423588077936640,@USAirways thanks for slowing the inbound plan...,1
5745,641469128990191616,And Argentina say Messi does nothing for the n...,1
2100,640575558313754625,It filled my heart with pride to see the huge ...,1
7705,569330782537437184,@JetBlue thanks for bringing my son home to me.,1


In [141]:
training_data.to_csv('../data/twitter/labelled_training_data.csv', index=False)

In [132]:
cols_to_use = ['Id', 'Author', 'Date', 'Text', 'Target']
training_data = pd.read_csv('../data/twitter/training/kaggle140tweets_data.csv'
                            , encoding = "ISO-8859-1"
                            # , nrows=1000
                            , header=None
                            , names=['Target', 'Id', 'Date', 'flag', 'Author', 'Text']
                           , usecols=cols_to_use)[cols_to_use]
training_data.groupby('Target').count()

,Id,Author,Date,Text
Target,,,,
0,800000,800000,800000,800000
4,800000,800000,800000,800000
